<a href="https://colab.research.google.com/github/MonishSoundarRaj/colab_files_1/blob/main/sdv_syn_theta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sdv

In [2]:
import pandas as pd
import numpy as np

In [3]:
theta_df = pd.read_csv("data_theta.csv")
theta_df.drop("Unnamed: 0", axis=1, inplace=True)
theta_df.head()

,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,71,3632.92,3600.0,8.073156,Killed
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,7772,21654.66,21600.0,1539.886933,Pass
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,340972,21655.97,21600.0,24639.681422,Killed
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,632522,3648.70,3600.0,4151.409778,Killed
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,23424,5995.18,10800.0,213.161956,Pass


In [4]:
from sdv.metadata import SingleTableMetadata

In [5]:
metadata = SingleTableMetadata()

In [6]:
metadata.detect_from_dataframe(data=theta_df)

In [7]:
print(metadata)

{
    "primary_key": "job",
    "columns": {
        "job": {
            "sdtype": "id"
        },
        "user": {
            "sdtype": "numerical"
        },
        "project": {
            "sdtype": "numerical"
        },
        "state": {
            "sdtype": "categorical"
        },
        "gpu_num": {
            "sdtype": "categorical"
        },
        "cpu_num": {
            "sdtype": "numerical"
        },
        "node_num": {
            "sdtype": "numerical"
        },
        "submit_time": {
            "sdtype": "numerical"
        },
        "wait_time": {
            "sdtype": "numerical"
        },
        "run_time": {
            "sdtype": "numerical"
        },
        "wall_time": {
            "sdtype": "numerical"
        },
        "node_hour": {
            "sdtype": "numerical"
        },
        "new_status": {
            "sdtype": "categorical"
        }
    },
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}


In [8]:
theta_df.head()

,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,639487,0.000000e+00,0.000000e+00,0,0,512.0,8.0,1672529301,71,3632.92,3600.0,8.073156,Killed
1,639448,0.000000e+00,0.000000e+00,0,0,16384.0,256.0,1672507846,7772,21654.66,21600.0,1539.886933,Pass
2,639135,0.000000e+00,0.000000e+00,0,0,262144.0,4096.0,1672174846,340972,21655.97,21600.0,24639.681422,Killed
3,638720,7.029633e+13,7.029633e+13,0,0,262144.0,4096.0,1671905072,632522,3648.70,3600.0,4151.409778,Killed
4,639463,0.000000e+00,0.000000e+00,0,0,8192.0,128.0,1672513950,23424,5995.18,10800.0,213.161956,Pass


In [9]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(
    metadata,
    enforce_rounding=False,
    epochs=30,
    verbose=True
)
synthesizer.fit(theta_df.head(14000))

synthetic_data = synthesizer.sample(num_rows=10000)

Gen. (0.46) | Discrim. (-0.47): 100%|██████████| 30/30 [01:13<00:00,  2.45s/it]


In [10]:
print(synthetic_data)

       job          user       project  state  gpu_num       cpu_num  \
0        0  3.337016e+13  2.662412e+13      0        0     64.000000   
1        1  7.799336e+13  8.370337e+12      0        0     64.000000   
2        2  8.443128e+13  3.281338e+13      0        0    224.708728   
3        3  0.000000e+00  0.000000e+00      0        0   9013.746185   
4        4  0.000000e+00  3.170189e+11      0        0  34084.626475   
...    ...           ...           ...    ...      ...           ...   
9995  9995  4.110088e+13  5.741233e+13      0        0    863.979547   
9996  9996  6.762901e+13  4.545776e+11      0        0   8429.252291   
9997  9997  6.872090e+13  8.173609e+13      0        0    135.703957   
9998  9998  3.729692e+13  0.000000e+00      0        0   8304.078688   
9999  9999  3.433613e+13  0.000000e+00      0        0   7815.918365   

        node_num  submit_time  wait_time     run_time     wall_time  \
0     145.680523   1678209318       4932   670.324549   3626.531

In [11]:
from google.colab import files
synthetic_data.to_csv("syn_ctgan_theta.csv")
files.download("syn_ctgan_theta.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
from sdv.single_table import TVAESynthesizer

synthesizer_TVAE = TVAESynthesizer(
    metadata,
    enforce_min_max_values=True,
    enforce_rounding=False,
    epochs=100,
)

synthesizer_TVAE.fit(theta_df.head(13000))
synthetic_data_tvae = synthesizer_TVAE.sample(num_rows=1000)


In [13]:
print(synthetic_data_tvae)

     job          user       project  state  gpu_num      cpu_num    node_num  \
0      0  8.153695e+13  8.181842e+13      0        0   247.610443    3.230759   
1      1  3.443197e+13  3.455009e+13      0        0   177.251760    2.682688   
2      2  8.000278e+13  8.009763e+13      0        0   473.455721    7.962741   
3      3  2.745529e+13  2.690516e+13      0        0  8093.674484  126.741049   
4      4  5.996715e+13  6.002412e+13      0        0   535.150361    8.148552   
..   ...           ...           ...    ...      ...          ...         ...   
995  995  9.541537e+09  0.000000e+00      0        0   506.088861    7.467492   
996  996  5.986160e+13  5.999431e+13      0        0  8185.028517  128.325955   
997  997  3.472699e+11  2.675386e+11      0        0   233.908218    4.115853   
998  998  5.993836e+13  6.001287e+13      0        0  8336.271035  129.309314   
999  999  7.200991e+13  7.206318e+13      0        0  8437.955860  131.304896   

     submit_time  wait_time

In [15]:
from google.colab import files
synthetic_data_tvae.to_csv("syn_tvae_theta_epochs100.csv")
files.download("syn_tvae_theta_epochs100.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>